In [181]:
import requests
import pandas as pd

In [182]:
df = pd.read_csv('https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2025-07/HomicideActuals1987-2024.csv')


/tmp/ipykernel_6441/1863618298.py:1: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2025-07/HomicideActuals1987-2024.csv')


In [183]:
# Keep only rows where incident year is 2015 or later
ca_homicide = df[df['Inc YR'] >= 2015]
#ca_homicide.info()

In [184]:
# Take out columns that I'm not gonna use
columns_to_remove = ['NCIC', 'BCS', 'Rpt MO', 'Rpt YR']
ca_homicide = ca_homicide.drop(columns=columns_to_remove)
ca_homicide.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19293 entries, 73125 to 92487
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CO              19293 non-null  int64  
 1   Vict num        14759 non-null  float64
 2   Tot vic         14759 non-null  float64
 3   Tot susp        14759 non-null  float64
 4   V sex           19288 non-null  float64
 5   V race          19293 non-null  object 
 6   V age           19293 non-null  object 
 7   Crm Stat        19293 non-null  int64  
 8   VO_1            19293 non-null  int64  
 9   VO_2            1275 non-null   float64
 10  VO_3            402 non-null    float64
 11  VO_4            144 non-null    float64
 12  Inc MO          19293 non-null  int64  
 13  Inc day         19293 non-null  int64  
 14  Inc YR          19293 non-null  int64  
 15  Week day        14759 non-null  float64
 16  Death YR        14759 non-null  float64
 17  Means of Death  19293 non-null  

In [185]:
# wanna know about homicides by county
homicides_by_county = ca_homicide.groupby('CO').size().reset_index(name='Total_Homicides_Case')
homicides_by_county = homicides_by_county.sort_values('Total_Homicides_Case', ascending=False)
homicides_by_county.head()

,CO,Total_Homicides_Case
17,19,6374
34,36,1379
0,1,1254
31,33,1020
35,37,921


In [186]:
county_naming = pd.read_csv('county_names.csv')
homicides_by_county = homicides_by_county.merge(county_naming, on='CO', how='left')
homicides_by_county = homicides_by_county[['CO', 'County_Name', 'Total_Homicides_Case']]
#homicides_by_county.head()

In [187]:
# Filter homicides falling under DV categories
# DV relationship codes: 1husband, 2wife, 3common-law husband, 4common-law wife, 22Boyfriend/Ex-Boyfriend
# 23Girlfriend/Ex-Girlfriend, 24Ex-husband,25Ex-wife, 29Homosexual relationship

dv_codes = [1,2,3,4,22,23,24,25,29]

dv_homicides = ca_homicide[
    ca_homicide[['VO_1']].isin(dv_codes).any(axis=1)
]

#dv_homicides.info()

In [188]:
print(f"{len(dv_homicides)} homicide cases over the past decade in California fall under the domestic violence category.")

1452 homicide cases over the past decade in California fall under the domestic violence category.


In [189]:
dv_homicides.to_csv("cadv_homicides_2015_2024.csv", index=False)

In [190]:
# to find out what types of Victim/Offender relationship are most common
ca_homicides_relationship = ca_homicide.groupby('VO_1').size().reset_index(name='VO_Relationship')
ca_homicides_relationship = ca_homicides_relationship.sort_values('VO_Relationship', ascending=False)
ca_homicides_relationship

,VO_1,VO_Relationship
30,50,9171
28,40,3351
18,21,1459
27,30,1215
20,23,583
2,2,503
29,45,489
25,28,398
16,16,270
5,5,269
